# 정규표현 응용: 내용 추출 및 변경

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - nltk book 3.Processing raw text(https://www.nltk.org/book/ch03.html) 내용을 참고해서 자료를 만듦. 
    - nltk book의 3장 예제 인용.

### 접두사 r에 관해
- 파이썬 interpreter 입장에서 보면 정규표현은 일반 문자열과 다를 바 없습니다. 
- 정규표현에 backslash(\\)가 포함된 메타기호(예: \b)가 있다면, 
    - interpreter에게 이 문자열은 re(regular expression) 라이브러리에게 처리를 맡기라고 알려줘야 합니다. 
- 이를 위해 알파벳 r을 정규표현 문자열 앞에 붙입니다.
    - r을 붙이지 않으면, interpreter가 backslash(\\) 의미를 모른 채 직접 처리하기 때문에 기대했던 결과를 얻지 못합니다. 
    - 습관처럼 r을 붙이면 이런 bug는 발생하지 않습니다.
- 참고로 backslash(\\)는 보통 escape character라고 부릅니다.    

In [ ]:
import nltk, re

### findall(): 매칭된 모든 단어 추출
아래 예제에서는 word에서 모음만 모두 찾거나, 모음이 2개 이상 있는 패턴을 찾아 보겠습니다.

In [ ]:
word = 'supercalifragilisticexpialidocious'
print(re.findall(r'[aeiou]', word))
print(len(re.findall(r'[aeiou]', word)))
print(re.findall(r'[aeiou]{2,}', word))

### treebank 말뭉치
- 미국 펜실배니아 주립대학(Univ. of Pennsylvania)에서 만든 말뭉치.
- 각 문장에 대한 구문 분석 결과가 포함된 말뭉치(parsed corpora).  참고사이트: https://www.nltk.org/howto/corpus.html 
    - nltk에서는 전체 말뭉치 중 10%만 포함.
- DT(Determiner, 관사), NN(Noun, singular, 명사-단수), IN(Preposition, 전치사), RB(Adverb, 부사) 등은 품사 이름. 
    - 참고사이트: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [ ]:
from nltk.corpus import treebank

print(treebank.fileids()[:5])
print(treebank.tagged_words('wsj_0003.mrg')[:7])
#print(treebank.parsed_sents('wsj_0003.mrg')[0]) 

- treebank 말뭉치에서 모든 품사의 빈도를 구해 보겠습니다.
    - 빈도 순으로 단수명사(NN), 전치사(IN), 복수명사(NNP), 관사(DT) 순입니다.
    - NONE은 품사를 알 수 없는 경우입니다.
- **NOTE**: **FreqDist()** 는 dictrionary 객체(key, value 쌍)의 리스트를 반환합니다.

In [ ]:
from nltk.probability import FreqDist

tag_list = [tag for word, tag in nltk.corpus.treebank.tagged_words()]
fd = FreqDist(tag_list)

print(fd.most_common(7)) #빈도 순으로 상위 7개만 출력
#print(fd.most_common()) #전체 출력

treebank 말뭉치에 속한 단어 중에서 모음이 연속해서 2번 이상 나타난 빈도(FreqDist)를 구해 보겠습니다.

In [ ]:
wsj = sorted(set(nltk.corpus.treebank.words()))
new_list = [vs for word in wsj 
            for vs in re.findall(r'[aeiou]{2,}', word)]

fd = FreqDist(new_list)
print(fd.most_common())

- re 라이브러리의 findall() 메소드 연습을 하던 중이었죠...
- 복습 차원에서 간단한 예를 다뤄 보죠. 년월일에서 **정수** 숫자만 찾아볼까요...

In [ ]:
[int(n) for n in re.findall(r'[0-9]+', '2021-05-31')]

int 타입으로 변환하지 않으면 어떻게 될까요?

In [ ]:
#[n for n in re.findall(r'[0-9]+', '2021-05-31')] 

- 단어에서 첫 번째 문자와 마지막 문자가 모음인 경우를 제외하고, 안쪽에 놓인 모음은 모두 제거하는 예입니다.
    - 예: declaration -> dclrtn(declaration: 선언), inalienable -> inlnble  (inablianable: 양도할 수 없는)
- 정규표현에서 3개의 패턴을 정의하고 이를 메타기호(|)를 사용하여 연결합니다.
    - 첫 번째 문자가 모음 | 마지막 문자가 모음 | 모음을 제외한 모든 문자

In [ ]:
def compress(word):
    regexp = r'^[aeiouAEIOU]|[aeiouAEIOU]$|[^aeiouAEIOU]'
    #regexp = r'[^aeiouAEIOU]|^[aeiouAEIOU]|[aeiouAEIOU]$' #순서를 바꿔도 될까요?
   
    pieces = re.findall(regexp, word)
    return ''.join(pieces) #찾은 문자들을 공백없이 하나의 문자열로 합침
    #return pieces #찾은 문자들을 리스트로 출력

In [ ]:
print(compress('declaration'))
print(compress('inalienable'))

### udhr(the Universal Declaration of Human Rights, 세계인권선언) 말뭉치
- 300여개 언어로 된 말뭉치를 갖고 있습니다. 
- 당연히 우리말도 있습니다. udhr의 fileids는 UTF8 이나 Latin1처럼 문자체계 인코딩 정보를 포함합니다.

In [ ]:
print(nltk.corpus.udhr.words('Korean_Hankuko-UTF8')[:15])

- **nltk.tokenwrap(tokens, separator=' ', width=70)** 
    - tokens를 70자씩(default) 묶어서 출력하는 함수입니다. 한 줄에 70자가 넘어가면 자동으로 줄을 바꿉니다.
        - wrap은 한 데 묶는다는 뜻입니다. wrapper 함수 많이 들어봤죠...단어 rap은 의미가 전혀 다릅니다. ^^~
    - 토큰과 토큰 사이에는 구분 기호(separator)로 빈 칸(default)이 포함됩니다. 

In [ ]:
english_udhr = nltk.corpus.udhr.words('English-Latin1')
print(nltk.tokenwrap(compress(w) for w in english_udhr[:75]))

### Rotokas 말뭉치
- rotokas어는 파푸아 뉴기니아의 Bougainville섬에서 사용하는 언어입니다.
- 남태평양의 작은 섬나라 언어까지 연구했다는 점에 경의를 표하고 싶습니다.

- rotokas어에서 사용되는 단어 중 자음-모음(2글자) 형태(예: ka, si)를 추출합니다. 
    - rotokas어의 자음은 [ptksvr], 모음은 [aeiou] 입니다. 
    - 확실치는 않지만, 영어 알파벳을 차용해서 rotokas 언어를 만든 것 같습니다.
- 조건빈도분석(**ConditionalFreqDist**)을 사용해 각 쌍의 빈도를 표 형태로 출력합니다. 

In [ ]:
rotokas_words = nltk.corpus.toolbox.words('rotokas.dic')
cvs = [cv for w in rotokas_words 
          for cv in re.findall(r'[ptksvr][aeiou]', w)]
cfd = nltk.ConditionalFreqDist(cvs)
cfd.tabulate()

- 자음 s와 모음 i에 주목할 필요가 있습니다. 
- Rotokas어는 뚜렷한 음소(phonemes)를 갖고 있지 않다는 점과 s의 경우 si로 거의 대부분 사용되고 있기 때문이죠.

### defaultdict( ) 
- dictionary 객체의 값이 컬렉션(lists, dicts, ...)인 경우, 이 dictionary 객체의 키를 사용해 값을 참조하려면, 
    - key 해당하는 값은 초기화(empty list 또는 empty dict)되어 있어야 합니다. 
    - 이런 초기화 작업을 defaultdict가 자동으로 처리해줍니다. 
- 예제 참조: https://www.accelebrate.com/blog/using-defaultdict-python    

- 리스트를 defaultdict(**int**)를 사용하여 dictionary 객체로 만듭니다. 
    - 이때 키(key)에 상응하는 값(value)은 모두 0으로 초기화됩니다.

In [ ]:
from collections import defaultdict

food_list = 'spam spam spam spam spam spam eggs spam'.split()
food_count = defaultdict(int) #int: 0으로 초기화
print(food_count['spam'], food_count['eggs'])

for food in food_list:
    food_count[food] += 1 
print(food_count)

### nltk.Index()
- 이번에는 defaultdict(**list**)를 사용하여 dictionary 객체(**cities_by_state**)를 만들면서 empty list로 초기화했습니다.
    - **city_list**는 튜플을 원소로 갖는 리스트입니다.
- defaultdict를 사용하여 dictionary 객체로 만들 수도 있지만,
    - **Index()**를 사용하면 이보다 간편하게 dictionary 객체로 만들 수 있습니다.

In [ ]:
city_list = [('TX', 'Austin'), ('TX', 'Houston'), ('NY', 'Albany'), ('NY', 'Syracuse'), 
             ('NY', 'Buffalo'), ('NY', 'Rochester'), ('TX', 'Dallas'), ('CA','Sacramento'), 
             ('CA', 'Palo Alto'), ('GA', 'Atlanta')]

In [ ]:
cities_by_state = defaultdict(list) #list: empty list로 초기화
for state, city in city_list:
     cities_by_state[state].append(city)
for state, cities in cities_by_state.items():
     print(state, ', '.join(cities))

nltk.Index()를 사용해서 dictionary 객체로 변환하기 위해서는 튜플 타입의 리스트여여 합니다.

In [ ]:
cities_by_state_index = nltk.Index(city_list)        
cities_by_state_index

- 위 표에 나타난 자음-모음 쌍에 해당하는 단어들을 찾으면, Rotokas 언어 분석에 훨씬 도움이 되겠죠...
- Index() 메소드를 사용하면 이를 간단히 구현할 수 있습니다. 

In [ ]:
cv_word_pairs = [(cv,w) for w in rotokas_words
                        for cv in re.findall(r'[kprstv][aeiou]', w)]

print(type(cv_word_pairs)) #list of tuples
cv_index = nltk.Index(cv_word_pairs) 
print(cv_index['su'])
print(cv_index['po'])

## 어간(word stem) 찾기
- 웹에서 laptop을 검색할 때 검색 결과에 laptops도 포함됩니다.
    - laptop, laptops는 모두 같은 어간인 laptop에서 파생되었기 때문입니다.
- 자연어처리에서는 단어 끝에 붙은 접미사 대신 어간을 구해 같은 단어를 중복해서 처리하지 않아야 합니다.
- NLTK에서 stemmer를 제공하고 있습니다(PorterStemmer, LancasterStemmer)
    - 단어의 뿌리에 해당하는 표제어를 찾아주는 WordNetLemmatizer도 있습니다. (is, are -> be)
- 아래처럼 직접 어간을 찾는 함수를 구현할 수도 있습니다.    

In [ ]:
def stem(word):
    for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

stem('processing')

여기서는 정규표현을 사용하여 어간을 찾는 연습을 해 보겠습니다.

In [ ]:
re.findall(r'^.*(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

- 실행 결과 정규표현이 단어 전부와 매칭되었지만, 접미사 ing만 출력되었습니다. 왜 그럴까요? 
    - 그 이유는 괄호를 사용했기 때문입니다. 
    - 괄호 표현은 패턴만 정의한 것이 아니라, 패턴에 해당하는 부분 문자열을 출력하라는 의미도 포함되어 있습니다. 
- 괄호를 여러 패턴의 조건을 묶는 목적으로만 사용하고 해당 패턴을 굳이 출력하지 않으려면,  '?:'를 붙이면 됩니다.

In [ ]:
re.findall(r'^.*(?:ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

- 이번에는 processing 전체가 매칭되어 모두 출력되었습니다.
- 조금 진지하게 문제에 접근해 볼까요...
    - 괄호를 2개 사용해서 어간(stem)과 접미사(suffix) 부분을 분할해 보겠습니다.

In [ ]:
re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

어간 추출이 잘 되는 것처럼 보이지만, 여전히 문제점이 숨어 있습니다. 다른 단어로 테스트해 보겠습니다.

In [ ]:
re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')

- 왜 이런 결과가 나왔을까요? 
    - 메타기호인 \*(star) 의 탐욕스런(greedy) 성질 때문입니다. 
    - \*는 최대한 많은 문자들과 매칭하려는 욕심쟁이 성질을 갖고 있습니다. 
- \*의 탐욕스런 성질을 멈추게 해야 접미사 패턴 es 가 매칭될 수 있는 기회를 갖게 되겠죠...
    - \*의 탐욕스런 성질을 멈추게 하는 방법은 \*?와 같이 ?기호를 추가하는 것입니다.

In [ ]:
re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')

개선된 정규표현은 제거할 접미사가 없는 경우에도 잘 동작합니다. 

In [ ]:
re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$', 'language')

- 앞에서 만든 stem() 함수를 정규표현을 사용한 함수로 변경하고 실전에 적용해 보겠습니다.
- 사실 NLTK에 내장된 stemmer(PorterStemmer, LancasterStemmer)도 완벽하게 어간을 찾지는 못합니다.

In [ ]:
def stem(word):
    regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
    stem, suffix = re.findall(regexp, word)[0] #리스트가 1개의 원소만 갖고 있음.
    return stem  #어간과 접미사 중 어간만 반환 

In [ ]:
from nltk import word_tokenize #토큰 찾기 모듈

raw = """
DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government.  Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony.
"""
tokens = word_tokenize(raw)

- 다양한 에러가 발생하였습니다. 
    - ponds, is, basis에서 무조건 s를 없애 엉뚱한 어간을 출력했군요. 
    - distributing와 derives도 각각 distribut와 deriv로 출력했습니다.

In [ ]:
print([stem(t) for t in tokens])

## 토큰(token)으로 정의한 단어 검색
- 예를 들면, "\< a\>  \< man\>" 은 텍스트에서 'a man' 이 나타난 구(phrase)를 찾습니다. 
    - 각괄호(angle bracket)는 토큰 경계를 표시합니다. 
    - 각괄호 사이의 whitespace는 무시됩니다.

### Gutenberg 말뭉치
- NLTK는 Gutenberg 프로젝트(http://www.gutenberg.org/)의 25,000여개 전자책(무료) 중 18개의 텍스트를 말뭉치로 제공 

- 첫 번째 예: 단일 토큰 \"\< a\>"와 "\< man\>" 사이에 (\<.*\>) 를 괄호로 묶었기 때문에, 이 부분에 해당하는 문자열만 출력된다. 
    - 예: a monied man -> monied

In [ ]:
from nltk.corpus import gutenberg
moby = nltk.Text(gutenberg.words('melville-moby_dick.txt'))
moby.findall(r"<a> (<.*>) <man>")

### nps_chat(채팅 말뭉치)
- 미국 해군 대학원(the Naval Postgraduate School)에서 인터넷 악성 공격을 탐지하기 위한 연구 목적으로 구축.
- 15개 파일과 10,000개 이상의 포스팅으로 구성됨. 

두 번째 예: 3개 단어로 이루어진 구(phrase)이며, 마지막 단어가 'bro'인 구를 찾습니다. 

In [ ]:
from nltk.corpus import nps_chat

chat = nltk.Text(nps_chat.words())
chat.findall(r"<.*> <.*> <bro>")

세 번째 예: 알파벳 'l'(엘)로 시작하는 3개 이상 단어로 이루어진 구를 찾습니다.

In [ ]:
chat.findall(r"<l.*>{3,}")

In [ ]:
#nltk.app.nemo() #원문에서 정규표현 패턴과 일치되는 문자열을 확인할 수 있습니다.

### Brown 말뭉치
- 미국 브라운 대학교(Brown University)에서 1961년에 최초로 만든 전자 말뭉치.
- 500개 이상의 source로부터 수집한 텍스트로 구성했으며, 뉴스, 종교, 취미, 미스테리 등 다양한 장르(genre)로 구분.

**Quiz 1**: 아래 정규표현이 찾으려는 구(phrase)의 특징은 무엇일까요?

In [ ]:
from nltk.corpus import brown
hobbies_learned = nltk.Text(brown.words(categories=['hobbies', 'learned']))
hobbies_learned.findall(r"<\w*> <and> <other> <\w*s>")

**Quiz 2**: 아래 정규표현이 찾으려는 구(phrase)의 특징은 무엇일까요?

In [ ]:
hobbies_learned.findall(r"<as> <\w*> <as> <\w*s>")